# Graphical User Interface for Single Tello Operation



### Imports and Globals
The first step is to import the required libraries...

In [1]:
# Standard library imports
import threading 
import socket
import queue
#from PIL import Image
from time import sleep


# Third party imports
import numpy as np
import cv2
from IPython.display import display
import ipywidgets.widgets as widgets

# Local application imports


### Tello Communication
The Tello uses WiFi for all of its communication.

The video socket is created by the OpenCV VideoCapture method defined below.
*** Important Note: teh current Tello firmware can only stream video if the Tello is directly connect to the host 
computer - i.e. at 192.168.10.1 without using a router. If you assign it to an existing netowork it will not stream video.
The streamon command will not return an error.

In [2]:
# Globals

#TelloIP = '10.10.30.31'
TelloIP = '192.168.10.1'

TelloCmdPort = 8889      # Command and response
TelloStatusPort = 8890   # Status data from the Tello 
TelloVideoPort = 11111   # h.264 video stream from the Tello

### Create a UDP server to listen for command respose messages from the Tello.
A UDP format message is not actually a client / server connect, but we still use that terminology. 

In [3]:
# Create a UDP socket for commands
cmd_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
cmd_socket.bind (('192.168.10.2', TelloCmdPort)) #host ip, port

# Create a UDP socket for status
status_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
status_socket.bind(('192.168.10.2', TelloStatusPort)) # host, port

# Create a UDP socket for video
#video_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
#video_socket.bind(('192.168.10.2', TelloVideoPort)) # host, port

### Define Background Threads for Tello Communication
Create threads to listen for responses to commands, collect status data and capture the video stream fromm the Tello

In [4]:
def receive_command_response_thread(_widget):

    while True: 
        try:
            data, ip = cmd_socket.recvfrom(1518)
            _widget.value = data.decode(encoding='utf-8')
            
        except Exception:
            print ('\nreceive_command_response exception - Exit . . .\n')
            break

In [5]:
# The status data is sent from the Tello in a fixed order. The background thread reads that data and builds a list
# of strings containing the data lables and values. 

def receive_status_thread(_status_items):

    while True: 
        try:
            b_data, ip = status_socket.recvfrom(1518)
            #print ("Got a status", ip, data)
            data = b_data.decode(encoding='utf-8')
            data = data[:-5].split(';') # convert the data into an array of individual strings, cutting off the trailing ';\r\n'

            # The elements defined in the GUI rely on the data being saved to the list in order as defined in the SDK
            # todo - test to make sure data lenght = list size
            for index in range (len(data)):
                _status_items[index].description = data[index].split(':')[0]
                _status_items[index].value = data[index].split(':')[1]              
            
            # Convert the status data byte array to a string and save it and the ip address to the queue
            #StatusQ.put(PacketBlob(data.decode(encoding="utf-8"), ip[0]))
            
        except Exception:
            print ('\nreceive_status exception - Exit . . .\n')
            break

### Load a Deep Neural Network for Image Processing
The ...

In [6]:
from NVidia.object_detection import *

model = ObjectDetector('ssd_mobilenet_v2_v04_coco.engine')

### Create a Graphical User Interface
The user intreface is built using Jupyter Widgets...

Explain event driven programming using on submit and observe methods...

#### Title Bar

In [7]:
title_bar = widgets.Text(description = "Interfaced to Tello ID - ")
title_bar.value = TelloIP

#### Status Data Area
The status data area is created using a grid widget. This widget allows all of the elements to be placed at uniformly
spaced intervals. A list of Text widgets is created (status_items) to hold each piece of status data from the Tello.
One widget from the list is then assigned a specific spot on the grid to create the data layout that is desired.
The status thread will directly update the values for each widget in the background. The order of the data in the list
is just the prder defined the the SDK. For example status_item[15] is the battery voltage.

In [8]:
status_grid = widgets.GridspecLayout (9,4)

style = {'description_width':'40px'}
status_items = [ widgets.Text(value = str(index),
                              description=str(index),
                              layout=widgets.Layout(width='100px'),
                              style=style, 
                              disabled=True) for index in range(21) ]

status_grid[0,0] = widgets.Label(value="Found MP:")
status_grid[0,0] = status_items[0]
                          
for i in [1,2,3]:
    status_grid[1,i] = status_items[i]

status_grid[2,0] = widgets.Label(value="Attitude:")
for i in [1,2,3]:
    status_grid[2,i] = status_items[i+4]
    
status_grid[3,0] = widgets.Label(value="Velocity:")
for i in [1,2,3]:
    status_grid[3,i] = status_items[i+7]

status_grid[4,0] = widgets.Label(value="Acc:")
for i in [1,2,3]:
    status_grid[4,i] = status_items[i+17]

status_grid[5,0] = widgets.Label(value="Temp:")
status_grid[5,1] = status_items[11]
status_grid[5,2] = status_items[12]
    
status_grid[6,0] = widgets.Label(value="Time:")
status_grid[6,1] = status_items[13]
status_grid[6,2] = status_items[17]

status_grid[7,0] = widgets.Label(value="Altitdue:")
status_grid[7,1] = status_items[14]
status_grid[7,2] = status_items[16] 
    
status_grid[8,0] = widgets.Label(value="Battery:")
status_grid[8,1] = status_items[15]


#### Video Window

In [9]:
# Tello's native video resolution is 720p 1280 x 720
video_frame = widgets.Image(format='jpeg', width=170, height=90, description='test')
video_frame = video_frame.from_file('test_image.jpg')
# widgets.Image

def execute(change):
    image = change['new']
    
    # compute all detected objects
    detections = model(image)
    
    # draw all detections on image
    for det in detections[0]:
        bbox = det['bbox']
        cv2.rectangle(image, (int(300 * bbox[0]), int(300 * bbox[1])), (int(300 * bbox[2]), int(300 * bbox[3])), (255, 0, 0), 2)
    
    # select detections that match selected class label
    matching_detections = [d for d in detections[0] if d['label'] == 1]
    
    # get detection closest to center of field of view and draw it
    #det = closest_detection(matching_detections)
    #if det is not None:
    #    bbox = det['bbox']
    #    cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 5)
    
    video_frame.value = bytes(cv2.imencode('.jpg', image)[1])

#### Machine Learning Controls

Image Capture
* Button
* Preview window

Training
* Epochs
* Progress
* Loss
* Accuracy
    
Inference
* Convlution...
* Regression...

In [10]:
captured_image = widgets.Image(format='jpeg', width='100', height='100')
captured_image = captured_image.from_file('test_image2.jpg')

In [11]:
epochs_widget = widgets.IntText(description='epochs', value=1)
eval_button = widgets.Button(description='evaluate')
train_button = widgets.Button(description='train')
loss_widget = widgets.FloatText(description='loss')
accuracy_widget = widgets.FloatText(description='accuracy')
progress_widget = widgets.FloatProgress(min=0.0, max=1.0, description='progress')

mlc = widgets.VBox([ captured_image,
                   epochs_widget,
                   eval_button,
                   loss_widget,
                   accuracy_widget,
                   progress_widget ])
#display(mlc)

#### Commnd Line

In [12]:
command_line = widgets.Text(description='Commands:',
                            value='',
                            placeholder='command takeoff land flip forward back left right up down cw ccw speed speed?',
                            layout = widgets.Layout(width='800px'))

# Send any command entered to the Tello
def on_command_entered(widget_triggered):    
    msg = widget_triggered.value
    msg = msg.encode(encoding="utf-8") 
    sent = cmd_socket.sendto(msg, (TelloIP, TelloCmdPort))
    #widget_triggered.value = ''
    
command_line.on_submit(on_command_entered)


#the response line
response_line = widgets.Textarea(description='Response:',
                                 value='',
                                 layout = widgets.Layout(width='800px'))

command_area = widgets.VBox([ command_line,response_line ])


### Create The Composit GUI
The AppLayout widget is used to organize the individual elements of the GUI. The widget is divided into five 
main areas - a header, a footer and three middle areas - left, center and right. Each of the sections defined above
are assigned to the AppLayout widget and then the composit GUI is displayed.

In [13]:
composit_gui = widgets.AppLayout(header = title_bar,
                                 left_sidebar = status_grid,
                                 center = video_frame,
                                 right_sidebar = mlc,
                                 pane_widths = [2,2,2],
                                 pane_heights = ['50px',3,1],
                                 footer = command_area)
display(composit_gui)

AppLayout(children=(Text(value='192.168.10.1', description='Interfaced to Tello ID - ', layout=Layout(grid_are…

### Start The Background Threads

In [14]:
response_thread = threading.Thread(target=receive_command_response_thread, args=(response_line,))
response_thread.start()

status_thread = threading.Thread(target=receive_status_thread, args=(status_items,))
status_thread.start()

In [15]:
from tello_camera import TelloCamera

In [16]:
# The Tello camera resoultion is (720,960,3) but SSD_mobilenet was trained on an image
# of size (300,300,3). Setting the width and height will force the camera class to 
# resize the image to be compatible with the SDD_mobilenet engine
camera = TelloCamera.instance(width=300, height=300)

In [17]:
camera.unobserve_all()
camera.observe(execute, names='value')

In [22]:
camera.restart()

In [19]:
captured_image.value = bytes(0)

In [ ]:
video_thread = threading.Thread(target=receive_video_stream, args=(video_image,))
video_thread.start()

### Commands to get started

Place the Tello in command mode by sending 
    command
You should receive an ok. Sometimes the thread misses this - todo - debug this

start the video stream by sending
    streamon
You should receive an ok and video should appear inthe center window. There is a slight lag in the video.

todo - clear the command and response boxes on reentry to the command box

In [ ]:
def _h264_decode(packet_data):
    """
    decode raw h264 format data from Tello

    :param packet_data: raw h264 data array

    :return: a list of decoded frame
    """
    res_frame_list = []
    decoder = libh264decoder.H264Decoder()
    
    frames = decoder.decode(packet_data)
    
    for framedata in frames:
        (frame, w, h, ls) = framedata
        if frame is not None:
            # print 'frame size %i bytes, w %i, h %i, linesize %i' % (len(frame), w, h, ls)

            frame = np.fromstring(frame, dtype=np.ubyte, count=len(frame), sep='')
            frame = (frame.reshape((h, ls / 3, 3)))
            frame = frame[:, :w, :]
            res_frame_list.append(frame)

    return res_frame_list

def receive_video_thread(_widget):
    """
    Listens for video streaming (raw h264) from the Tello.

    Runs as a thread, sets self.frame to the most recent frame Tello captured.

    """
    packet_data = ""
    while True:
        try:
            res_string, ip = video_socket.recvfrom(2048)
            packet_data += res_string
            # end of frame
            if len(res_string) != 1460:
                for _frame in _h264_decode(packet_data):
                    frame = _frame
                packet_data = ""
            _widget.value = Image.fromarray(frame)

        except socket.error as exc:
            print ("Caught exception socket.error : %s" % exc)

In [ ]:
#vid = widgets.Video(value = 'udp//192.168.10.1:11111'.encode('utf-8'), format = 'url')

In [ ]:
cv2.__version__

In [ ]:
import imp

imp.find_module('ObjectDetector')

import os
import sys
import inspect

inspect.getfile(parse_boxes)

print('\n'.join(sys.path))

In [ ]:
import numpy as np
import cv2

In [ ]:
#cap = cv2.VideoCapture('udp://192.168.10.1:11111')
cap = cv2.VideoCapture('udp://0.0.0.0:11111',cv2.CAP_FFMPEG)

if not cap.isOpened():
    print('VideoCapture not opened')


In [ ]:
ret, frame = cap.read()

In [ ]:
frame.shape

In [ ]:
frame.dtype

In [ ]:
def receive_video_stream(_widget):

    cap = cv2.VideoCapture('udp://0.0.0.0:11111')

    #def bgr8_to_jpeg(value, quality=75):
    #return bytes(cv2.imencode('.jpg', value)[1])
    
    
    #frame = cap.read()
    #cap.release()

    while True:
        ret, frame = cap.read()
        
        _widget.value = bytes(cv2.imencode('.jpg', frame)[1])
        #cv.imshow('DJI Tello', frame)

        # Video Stream is closed if escape key is pressed
        #k = cv.waitKey(1) & 0xFF
        #if k == 27:
            #break
    #cap.release()
    #cv.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture('udp://0.0.0.0:11111',cv2.CAP_FFMPEG)
re, frame = cap.read()

In [ ]:
re

In [ ]:
frame.shape

In [ ]:
video_image.value = bytes(cv2.imencode('.jpg', frame)[1])

In [ ]:
frame.size

In [ ]:
rframe = np.empty((720,960,3), dtype=np.uint8)

In [ ]:
rframe.shape

In [ ]:
rframe = cv2.resize(frame, (300,300),0,0,interpolation=cv2.INTER_AREA)

In [ ]:
rframe.shape